In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn as sns
import warnings

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [4]:
data_dir = Path('./data/')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
n_fold = 10
n_class = 3
seed = 42

In [5]:
algo_name = 'lrcv'
feature_name = 'polyfeature'
model_name = f'{algo_name}_{feature_name}'

feature_file = data_dir / f'{feature_name}.csv'
p_val_file = data_dir / f'{model_name}.val.csv'
p_tst_file = data_dir / f'{model_name}.tst.csv'
sub_file = data_dir / f'{model_name}.csv'

In [6]:
df = pd.read_csv(data_dir / 'feature.csv', index_col=0)
print(df.shape)
df.head()

(400000, 25)


,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,class,d_dered_u,d_dered_g,d_dered_r,d_dered_i,d_dered_z,d_dered_zi,d_dered_zr,d_dered_zg,d_dered_zu,d_dered_ir,d_dered_ig,d_dered_iu,d_dered_ru,d_dered_gu,d_obs_det
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,2.9444,1.1898,0.0,-0.1397,-0.0790,-0.0544,-0.0403,-0.0307,-0.7232,-2.0462,-3.3488,-6.2153,-1.3230,-2.6257,-5.4921,-4.1691,-2.8665,6.1132
1,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,0.6931,1.2533,1.0,-0.0857,-0.0574,-0.0410,-0.0322,-0.0343,-0.1016,-0.2767,-0.8698,-1.8317,-0.1751,-0.7683,-1.7302,-1.5550,-0.9619,1.4427
2,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,1.0986,1.0225,0.0,-0.1787,-0.1388,-0.0963,-0.0718,-0.0540,-0.0257,-0.1440,-0.3905,-1.3116,-0.1183,-0.3649,-1.2859,-1.1676,-0.9211,1.8205
3,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,1.6094,1.2054,0.0,-0.3070,-0.1941,-0.1339,-0.1003,-0.0795,-0.2395,-0.5321,-1.4395,-5.7985,-0.2926,-1.2000,-5.5590,-5.2664,-4.3590,1.8640
4,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,2.6391,1.1939,0.0,-0.6820,-0.2653,-0.1794,-0.1339,-0.1067,-0.3144,-0.9871,-2.5579,-5.8955,-0.6727,-2.2436,-5.5811,-4.9084,-3.3376,4.5471


In [7]:
scaler = StandardScaler()
poly = PolynomialFeatures(2)
X = poly.fit_transform(scaler.fit_transform(df.drop(target_col, axis=1)))
feature_names = poly.get_feature_names(df.columns)
feature_names

['1',
 'z',
 'redshift',
 'dered_u',
 'dered_g',
 'dered_r',
 'dered_i',
 'dered_z',
 'nObserve',
 'airmass_u',
 'class',
 'd_dered_u',
 'd_dered_g',
 'd_dered_r',
 'd_dered_i',
 'd_dered_z',
 'd_dered_zi',
 'd_dered_zr',
 'd_dered_zg',
 'd_dered_zu',
 'd_dered_ir',
 'd_dered_ig',
 'd_dered_iu',
 'd_dered_ru',
 'd_dered_gu',
 'z^2',
 'z redshift',
 'z dered_u',
 'z dered_g',
 'z dered_r',
 'z dered_i',
 'z dered_z',
 'z nObserve',
 'z airmass_u',
 'z class',
 'z d_dered_u',
 'z d_dered_g',
 'z d_dered_r',
 'z d_dered_i',
 'z d_dered_z',
 'z d_dered_zi',
 'z d_dered_zr',
 'z d_dered_zg',
 'z d_dered_zu',
 'z d_dered_ir',
 'z d_dered_ig',
 'z d_dered_iu',
 'z d_dered_ru',
 'z d_dered_gu',
 'redshift^2',
 'redshift dered_u',
 'redshift dered_g',
 'redshift dered_r',
 'redshift dered_i',
 'redshift dered_z',
 'redshift nObserve',
 'redshift airmass_u',
 'redshift class',
 'redshift d_dered_u',
 'redshift d_dered_g',
 'redshift d_dered_r',
 'redshift d_dered_i',
 'redshift d_dered_z',
 'red

In [ ]:
df_poly = pd.DataFrame(data=X, columns=feature_names, index=df.index)
df_poly[target_col] = df[target_col]
df_poly.head()
df_poly.to_csv(feature_file)

In [12]:
del df_poly, df

In [13]:
df = pd.read_csv(feature_file, index_col=0)
print(df.shape)
df.head()

(400000, 325)


,1,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,class,d_dered_u,d_dered_g,d_dered_r,d_dered_i,d_dered_z,d_dered_zi,d_dered_zr,d_dered_zg,d_dered_zu,d_dered_ir,d_dered_ig,d_dered_iu,d_dered_ru,d_dered_gu,z^2,z redshift,z dered_u,z dered_g,z dered_r,z dered_i,z dered_z,z nObserve,z airmass_u,z class,z d_dered_u,z d_dered_g,z d_dered_r,z d_dered_i,z d_dered_z,z d_dered_zi,z d_dered_zr,z d_dered_zg,z d_dered_zu,z d_dered_ir,z d_dered_ig,z d_dered_iu,z d_dered_ru,z d_dered_gu,redshift^2,...,d_dered_z d_dered_ir,d_dered_z d_dered_ig,d_dered_z d_dered_iu,d_dered_z d_dered_ru,d_dered_z d_dered_gu,d_dered_zi^2,d_dered_zi d_dered_zr,d_dered_zi d_dered_zg,d_dered_zi d_dered_zu,d_dered_zi d_dered_ir,d_dered_zi d_dered_ig,d_dered_zi d_dered_iu,d_dered_zi d_dered_ru,d_dered_zi d_dered_gu,d_dered_zr^2,d_dered_zr d_dered_zg,d_dered_zr d_dered_zu,d_dered_zr d_dered_ir,d_dered_zr d_dered_ig,d_dered_zr d_dered_iu,d_dered_zr d_dered_ru,d_dered_zr d_dered_gu,d_dered_zg^2,d_dered_zg d_dered_zu,d_dered_zg d_dered_ir,d_dered_zg d_dered_ig,d_dered_zg d_dered_iu,d_dered_zg d_dered_ru,d_dered_zg d_dered_gu,d_dered_zu^2,d_dered_zu d_dered_ir,d_dered_zu d_dered_ig,d_dered_zu d_dered_iu,d_dered_zu d_dered_ru,d_dered_zu d_dered_gu,d_dered_ir^2,d_dered_ir d_dered_ig,d_dered_ir d_dered_iu,d_dered_ir d_dered_ru,d_dered_ir d_dered_gu,d_dered_ig^2,d_dered_ig d_dered_iu,d_dered_ig d_dered_ru,d_dered_ig d_dered_gu,d_dered_iu^2,d_dered_iu d_dered_ru,d_dered_iu d_dered_gu,d_dered_ru^2,d_dered_ru d_dered_gu,d_dered_gu^2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0002,-0.1093,1.8170,0.1270,0.9086,0.0208,0.0011,1.6087,0.1200,0.0,0.0063,0.1783,-0.0002,0.0008,-0.0156,-0.0367,-0.0626,-0.0975,-0.0389,-0.0981,-0.1349,-2.0425,-0.0928,1.4948,3.4181e-08,-2.0203e-05,0.0003,2.3488e-05,0.0002,3.8455e-06,2.0594e-07,0.0003,2.2191e-05,2.2235e-05,1.1669e-06,3.2960e-05,-3.7490e-08,1.4942e-07,-2.8794e-06,-6.7832e-06,-1.1575e-05,-1.8022e-05,-7.1979e-06,-1.8144e-05,-2.4946e-05,-0.0004,-1.7150e-05,0.0003,0.0119,...,1.5285e-03,2.1015e-03,0.0318,1.4447e-03,-0.0233,1.3461e-03,2.2970e-03,0.0036,1.4284e-03,3.6007e-03,0.0050,0.0749,0.0034,-0.0548,3.9195e-03,0.0061,2.4374e-03,6.1442e-03,0.0084,0.1279,0.0058,-0.0936,0.0095,0.0038,0.0096,0.0132,0.1991,0.0090,-0.1457,1.5157e-03,3.8209e-03,0.0053,0.0795,0.0036,-0.0582,9.6316e-03,0.0132,0.2005,0.0091,-0.1467,0.0182,0.2756,0.0125,-0.2017,4.1718,0.1895,-3.0532,0.0086,-0.1387,2.2346
1,1.0,-0.0719,-0.1007,-2.4251,-0.2656,-2.8534,-0.1751,-0.1054,-0.8761,0.6658,1.0,0.0081,0.2144,0.0001,0.0007,0.0071,0.0133,0.0156,0.0262,0.0123,0.0190,0.0327,0.4348,0.0274,-0.6478,5.1738e-03,7.2464e-03,0.1744,1.9104e-02,0.2052,1.2597e-02,7.5809e-03,0.0630,-4.7893e-02,-1.4200e-02,-5.8349e-04,-1.5419e-02,-8.7153e-06,-5.1123e-05,-5.0978e-04,-9.5460e-04,-1.1239e-03,-1.8852e-03,-8.8495e-04,-1.3637e-03,-2.3546e-03,-0.0313,-1.9677e-03,0.0466,0.0101,...,1.3437e-04,2.3200e-04,0.0031,1.9389e-04,-0.0046,1.7613e-04,2.0736e-04,0.0003,1.6328e-04,2.5161e-04,0.0004,0.0058,0.0004,-0.0086,2.4412e-04,0.0004,1.9223e-04,2.9622e-04,0.0005,0.0068,0.0004,-0.0101,0.0007,0.0003,0.0005,0.0009,0.0114,0.0007,-0.0170,1.5137e-04,2.3325e-04,0.0004,0.0053,0.0003,-0.0080,3.5943e-04,0.0006,0.0082,0.0005,-0.0123,0.0011,0.0142,0.0009,-0.0212,0.1891,0.0119,-0.2817,0.0007,-0.0177,0.4196
2,1.0,-0.0302,-0.1082,-1.5717,-0.1600,-1.4768,-0.0822,-0.0444,-0.4286,-1.3179,0.0,0.0013,0.0650,-0.0015,0.0002,0.0099,0.0170,0.0308,0.0409,0.0148,0.0444,0.0525,0.8020,0.0299,-0.4744,9.1298e-04,3.2708e-03,0.0475,4.8338e-03,0.0446,2.4830e-03,1.3416e-03,0.0130,3.9822e-02,-1.9476e-03,-3.9487e-05,-1.9652e-03,4.4024e-05,-5.3424e-06,-2.9778e-04,-5.1426e-04,-9.2915e-04,-1.2354e-03,-4.4839e-04,-1.3413e-03,-1.5874e-03,-0.0242,-9.0443e-04,0.0143,0.0117,...,4.3748e-04,5.1773e-04,0.0079,2.9499e-04,-0.0047,2.8967e-04,5.2337e-04,0.0007,2.5257e-04,7.5553e-04,0.0009,0.0136,0.0005,-0.0081,9.4561e-04,0.0013,4.5633e-04,1.3651e-03,0.0016,0.0247,0.0009,-0.0146

In [14]:
y = df[target_col].values[:320000]
df.drop(target_col, axis=1, inplace=True)
trn = df.iloc[:320000].values
tst = df.iloc[320000:].values
feature_name = df.columns.tolist()
print(y.shape, trn.shape, tst.shape)

(320000,) (320000, 324) (80000, 324)


In [15]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [16]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = LogisticRegression(multi_class='multinomial')
    clf.fit(trn[i_trn], y[i_trn])
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5
training model for CV #6
training model for CV #7
training model for CV #8
training model for CV #9
training model for CV #10


In [17]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

90.0009%


In [18]:
print(p_val.shape, p_tst.shape)

(320000, 3) (80000, 3)


In [19]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [20]:
clf.coef_.shape

(3, 324)

In [21]:
imp = pd.DataFrame({'feature': df.columns, 'importance': clf.coef_[1, :].T})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh', figsize=(64, 256))

<AxesSubplot:ylabel='feature'>

In [22]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [23]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [24]:
sub[target_col].value_counts()

2    42643
0    30879
1     6478
Name: class, dtype: int64

In [25]:
sub.to_csv(sub_file)